# Global params

In [1]:
img_directory = '/home/ec2-user/img_align_celeba'
img_save_directory = '/home/ec2-user/autoencoder-vaegan/zach'
model_weights_path = '/home/ec2-user/model-weights.hdf5'
batch_size = 64
training_set_size = 4096
img_size = 128

# Imports and helper functions

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
from utils import load_img, resize_crop
%matplotlib inline

def imshow(imgs):
    fig = plt.figure()
    nimgs = len(imgs)
    for i in range(nimgs):
        a=fig.add_subplot(1,nimgs,i+1)
        plt.imshow(imgs[i])
        plt.axis('off')

# Load training data

In [3]:
training = np.array([resize_crop(load_img(i+1, img_directory), (img_size, img_size)) for i in range(training_set_size)])

/home/ec2-user/.conda/envs/keras/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [4]:
imshow([training[i] for i in range(3)])

# Create model and load weights

In [14]:
from model import Vaegan
from keras.optimizers import Adam

model = Vaegan(img_shape=(img_size, img_size, 3), batch_size=batch_size)
model.model.compile(Adam(lr=.00003), 'binary_crossentropy')

In [7]:
if (False):
    model.model.load_weights(model_weights_path)

# Zach and latest output of Zach

In [8]:
onlyfiles = [f for f in os.listdir(img_save_directory) if os.path.isfile(os.path.join(img_save_directory, f))]
if (len(onlyfiles) > 0):
    latest = max(onlyfiles)
    imshow([training[0], sp.ndimage.imread(os.path.join(img_save_directory, latest))])

# Train

Reset image save filename counter

In [15]:
save_idx = 0

In [ ]:
# save an image of prediction before training
predictions = model.model.predict(training[:batch_size], batch_size=batch_size)
path = os.path.join(img_save_directory, '%04d.jpg' % save_idx)
sp.misc.imsave(path, predictions[0])
save_idx += 1

start = save_idx
stop = save_idx + 100

for i in range(start, stop):
    model.model.fit(x=training, y=training, epochs=1, batch_size=batch_size, verbose=True)
    predictions = model.model.predict(training[:batch_size], batch_size=batch_size)
    path = os.path.join(img_save_directory, '%04d.jpg' % i)
    sp.misc.imsave(path, predictions[0])
    model.model.save_weights(model_weights_path)
    save_idx = i

Epoch 1/1
2624/4096 [==================>...........] - ETA: 10s - loss: 24.9393

In [ ]:
predictions = model.model.predict(training[:batch_size], batch_size=batch_size)
predictions.shape
predictions[0]
# np.argwhere(np.isnan(predictions))